We are just going to add some test text here

In [12]:
import pandas as pd
import os
import numpy as np
import pyensembl


ModuleNotFoundError: No module named 'pyensembl'

In [2]:
# Get the metadata. This will be used to get the important genes from each supercluster
# The goal will be to take each supercluster and run a coloc for each gene in that with similar eqtls and GWAS 
# for each disease
metadata_df = pd.read_excel('/Users/nicholasjohnson/Desktop/workdir/magma/cluster_metadata.xlsx')

In [39]:
metadata_df['Top Enriched Genes']

0      IGHM, MS4A1, FCRL1, AC244205.1, IGLL5, IGHA1, ...
1      CD2, IL7R, PTPRC, SLFN12L, IL32, CCL5, GRAP2, ...
2      NKG7, GNLY, PRF1, KLRF1, PYHIN1, CCL4, RUNX3, ...
3      FCN1, CLEC12A, S100A4, TNFRSF1B, IFI30, LYZ, C...
4      SRGN, RGS1, GPR183, CD69, HLA-DRA, OLR1, TNFRS...
                             ...                        
457    LINC01036, SPTLC3, AC105213.1, SHOX2, LINC0092...
458    SPTLC3, NTS, COL6A5, AC105213.1, AL033530.1, C...
459    SLC15A1, COL6A5, LHX9, FSHR, LINC01997, SLITRK...
460    LINC01239, DMRTA1, MUC7, AC092445.1, AL159156....
461                                                  NaN
Name: Top Enriched Genes, Length: 462, dtype: object

In [4]:
# Reduce the map from LD reference to windows of 500kb around a gene, note positions are gr38 via NCBI.
# This test case will be SRGN on chr 10
LD_ref_all_df = pd.read_csv("/Users/nicholasjohnson/Desktop/workdir/magma/1kg_eur_1pct_ref_panel_lifted38_ld_reference.bim", delim_whitespace=True, engine='c', names=['CHR','SNP','CM','BP','MINOR_ALLELE','MAJOR_ALLELE'])


top_genes_per_cluster = metadata_df['Top Enriched Genes'].apply(lambda x: str(x).split(','))
idx = metadata_df['Supercluster'] == 'Microglia'
microglia_genes = top_genes_per_cluster[idx]
microglia_genes

4     [SRGN,  RGS1,  GPR183,  CD69,  HLA-DRA,  OLR1,...
5     [HLA-DRA,  TTR,  CD74,  HLA-DRB1,  SRGN,  RGS1...
6     [LNCAROD,  ITGAX,  CD74,  LINC00278,  FYB1,  A...
7     [CD74,  CX3CR1,  APBB1IP,  HLA-DRA,  LNCAROD, ...
8     [CCDC26,  LNCAROD,  FYB1,  CSF1R,  APBB1IP,  C...
9     [AIF1,  HLA-DRA,  CD74,  C1QC,  C1QB,  CX3CR1,...
10    [LNCAROD,  APBB1IP,  CX3CR1,  FYB1,  CSF1R,  C...
11    [CSF1R,  CX3CR1,  APBB1IP,  FYB1,  AL392172.1,...
12    [MRC1,  CD163,  STAB1,  MS4A6A,  MS4A4A,  F13A...
Name: Top Enriched Genes, dtype: object

In [36]:
    # CAn just make one large file of this for all info and import it later
    
    BP_window_low = 69087603 - 500000
    BP_window_high = 69104811 + 500000
    LD_ref_SGRN500kb_df = LD_ref_all_df[(LD_ref_all_df['CHR'] == 10) & (LD_ref_all_df['BP'] >= BP_window_low) & (LD_ref_all_df['BP'] <= BP_window_high)]

In [32]:
len(LD_ref_SGRN500kb_df)

3418

In [6]:
AD_gwas_df = pd.read_csv(os.path.abspath('/Volumes/CARD/projects/omicSynth/outcome_summary_stats/AD.ma'),  delim_whitespace=True, engine='c', header=0, names=['SNP','A1_AD','A2_AD','freq_AD','b_AD','se_AD','p_AD','N_AD'])
LBD_gwas_df = pd.read_csv(os.path.abspath('/Volumes/CARD/projects/omicSynth/outcome_summary_stats/LBD.ma'),  delim_whitespace=True, engine='c', header=0, names=['SNP','A1_LBD','A2_LBD','freq_LBD','b_LBD','se_LBD','p_LBD','N_LBD'])
PD_gwas_df = pd.read_csv(os.path.abspath('/Volumes/CARD/projects/omicSynth/outcome_summary_stats/PD.ma'),  delim_whitespace=True, engine='c', header=0, names=['SNP','A1_PD','A2_APD','freq_PD','b_PD','se_PD','p_PD','N_PD'])

In [21]:
AD_gwas_df.to_csv("/Users/nicholasjohnson/Desktop/workdir/magma/AD_gwas_summary_stats.ma")
PD_gwas_df.to_csv("/Users/nicholasjohnson/Desktop/workdir/magma/PD_gwas_summary_stats.ma")
LBD_gwas_df.to_csv("/Users/nicholasjohnson/Desktop/workdir/magma/LBD_gwas_summary_stats.ma")

In [34]:
# Merge with reference to reduce.
LD_with_AD = LD_ref_SGRN500kb_df.merge(AD_gwas_df, how='left', on='SNP')
#LD_with_AD_PD = LD_with_AD.merge(PD_gwas_df, how='left', on='SNP')
#merged_df = LD_with_AD_PD.merge(LBD_gwas_df, how='left', on='SNP')

In [35]:
len(LD_with_AD)

3418

In [10]:
# Make MAF effect and MAF columns.
merged_df['b_AD_MA'] = np.where(merged_df['freq_AD'] <= 0.50, merged_df['b_AD'], merged_df['b_AD']*-1)
merged_df['MAF_AD'] = np.where(merged_df['freq_AD'] <= 0.50, merged_df['freq_AD'], 1- merged_df['freq_AD'])
merged_df['b_PD_MA'] = np.where(merged_df['freq_PD'] <= 0.50, merged_df['b_PD'], merged_df['b_PD']*-1)
merged_df['MAF_PD'] = np.where(merged_df['freq_PD'] <= 0.50, merged_df['freq_PD'], 1- merged_df['freq_PD'])
merged_df['b_LBD_MA'] = np.where(merged_df['freq_LBD'] <= 0.50, merged_df['b_LBD'], merged_df['b_LBD']*-1)
merged_df['MAF_LBD'] = np.where(merged_df['freq_LBD'] <= 0.50, merged_df['freq_LBD'], 1- merged_df['freq_LBD'])

In [11]:
# Export and summarize.
merged_df.to_csv("GWASs_SGRN500kb_region_extracted.csv", index=False)

In [79]:
str(start_byte)

"(3, b'\\xf7\\xff\\xff\\xffV<\\x07\\x00\\xd0\\x07\\x00', -9, b'\\xf7', b'\\xff', nan, nan, nan, nan)"